# Atividade 5 - Stratified K-Fold

Implementar uma função que FARÁ uma estimativa de erro de um classificador baseada em k-fold cross-validation

• Por enquanto, a função deve receber como parâmetros:
    - um conjunto de dados (amostra original)
    - o valor de k 
        e deve dividir a amostra original em k subconjuntos de amostras (treinamento e teste)
    - Futuramente, essa função vai estimar o erro de um classificador
    
Observações:

    – Cada parte deve manter a proporção de cada classe (validação cruzada estratificada) o máximo possível
    – Cada parte deve ter tamanho diferente de outra em no máximo 1 elemento em cada classe
        Ex: k = 3, Dataset: 80 positivas e 40 negativas (66% x 33%)
        Fold 1: Pos: 27, Neg: 14, Total: 41, Proporção: 66%;33%
        Fold 2: Pos: 27, Neg: 13, Total: 40, Proporção: 67%;32%
        Fold 3: Pos: 26, Neg: 13, Total: 39, Proporção: 66%;33%
    – Imprimir quantos elementos ficaram em cada parte (quantos de cada classe e quantos no total) e a proporção do fold    


In [2]:
from pandas import read_csv
from sklearn.model_selection import StratifiedKFold
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html

# Balanceamento das classes
from imblearn.combine import SMOTETomek

import warnings

warnings.filterwarnings("ignore")

random_state=5007

In [3]:
# importa dataset já pre-processado

df = read_csv('../data/df_processed.csv', index_col=0)

print(df.shape)

df.head()

(858, 36)


,Age,Number of sexual partners,First sexual intercourse,Num of pregnancies,Smokes,Smokes (years),Smokes (packs/year),Hormonal Contraceptives,Hormonal Contraceptives (years),IUD,...,STDs: Time since first diagnosis,STDs: Time since last diagnosis,Dx:Cancer,Dx:CIN,Dx:HPV,Dx,Hinselmann,Schiller,Citology,Biopsy
0,18,4.0,15.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,3.0,0,0,0,0,0,0,0,0
1,15,1.0,14.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,3.0,0,0,0,0,0,0,0,0
2,34,1.0,17.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,3.0,0,0,0,0,0,0,0,0
3,52,5.0,16.0,4.0,1.0,37.0,37.0,1.0,3.0,0.0,...,4.0,3.0,1,0,1,0,0,0,0,0
4,46,3.0,21.0,4.0,0.0,0.0,0.0,1.0,15.0,0.0,...,4.0,3.0,0,0,0,0,0,0,0,0


## Stratified Cross- Validation

In [4]:
# Cross Validation com dataset DESBALANCEADO

def stratified_k_fold(df, k, random_state, shuffle=False):
    # Utilizando todas as features como preditoras
    X = df.drop('Biopsy', axis=1)
    y = df['Biopsy']
    # quantidade original de classes
    count_classes = y.value_counts()
    # Stratified K Fold
    skf = StratifiedKFold(n_splits=k, shuffle=shuffle, random_state=random_state)
    skf.get_n_splits(X, y)
    print('k = {}, Dataset {} positivas e {} negativas ({:.2f}% x {:.2f}%)'.format(k, count_classes[0], 
                                                                                 count_classes[1], 
                                                                                 ((count_classes[0]/len(y))*100), 
                                                                                 ((count_classes[1]/len(y))*100)))
    
    # Scores (das futuras metricas)
    scores = []

    # Folds
    for fold in enumerate(skf.split(X, y)):
        fold_number = fold[0]+1
        train_index = fold[1][0] # train
        test_index = fold[1][1] # test
        # quantidade de classes dentro da fold
        count_classes_fold = y.iloc[test_index].value_counts()
        # proporções
        prop_pos = ((count_classes_fold[0]/count_classes_fold.sum())*100)
        prop_neg = ((count_classes_fold[1]/count_classes_fold.sum())*100)
        print('Fold {}: Pos: {}, Neg: {}, Total: {}, Proporção: {:.2f}% x {:.2f}%'.format(fold_number, 
                                                                            count_classes_fold[0],
                                                                            count_classes_fold[1], 
                                                                            count_classes_fold.sum(),
                                                                            prop_pos, prop_neg))
        # Fit do modelo (utilizando dados balanceados)
        #model_obj = model(**params).fit(X_train, y_train)
        # Score do model (utilizados dados nao-balanceados) - dados de teste
        #score = recall_score(y_test, model_obj.predict(X_test))
        #scores.append(score)
    #return np.array(scores)


In [5]:
# Cross Validation com dataset BALANCEADO (SMOTETomek)

def stratified_k_fold_SMOTE(df, k, random_state, shuffle=False):
    # Utilizando todas as features como preditoras
    X = df.drop('Biopsy', axis=1)
    y = df['Biopsy']
    # quantidade original de classes
    count_classes = y.value_counts()
    # Stratified K Fold
    skf = StratifiedKFold(n_splits=k, shuffle=shuffle, random_state=random_state)
    skf.get_n_splits(X, y)
    print('k = {}, Dataset (desbalanceado) {} positivas e {} negativas ({:.2f}% x {:.2f}%)'.format(k, count_classes[0], 
                                                                                 count_classes[1], 
                                                                                 ((count_classes[0]/len(y))*100), 
                                                                                 ((count_classes[1]/len(y))*100)))
    # SMOTETomek
    cc = SMOTETomek(random_state=random_state)

    # Scores (das futuras metricas)
    scores = []

    # Folds
    #for fold in enumerate(skf.split(X, y)):
    for fold, (train_index, test_index) in enumerate(skf.split(X, y), 1):
        fold_number = fold
        X_train, y_train = X.iloc[train_index], y.iloc[train_index]
        X_test, y_test = X.iloc[test_index], y.iloc[test_index]

        # SMOTETomek (apenas os dados de treino)
        print('\tBalanceando dados de treino fold {}...'.format(fold_number))
        X_train, y_train = cc.fit_resample(X_train, y_train)
        print('\t\tFold = {}, Dataset (balanceado) {} positivas e {} negativas ({:.2f}% x {:.2f}%)'.format(fold_number, 
                                                                                 y_train.value_counts()[0], 
                                                                                 y_train.value_counts()[1], 
                                                                                 ((y_train.value_counts()[0]/len(y_train))*100), 
                                                                                 ((y_train.value_counts()[1]/len(y_train))*100)))
        # quantidade de classes dentro da fold
        count_classes_fold = y_test.value_counts()
        # proporções
        prop_pos = ((count_classes_fold[0]/count_classes_fold.sum())*100)
        prop_neg = ((count_classes_fold[1]/count_classes_fold.sum())*100)
        print('\t\tDados de teste (desbalanceados)')
        print('\t\tFold {}: Pos: {}, Neg: {}, Total: {}, Proporção: {:.2f}% x {:.2f}%'.format(fold_number, 
                                                                            count_classes_fold[0],
                                                                            count_classes_fold[1], 
                                                                            count_classes_fold.sum(),
                                                                            prop_pos, prop_neg))
#         print('----'*25)
        
        # Fit do modelo (utilizando dados balanceados)
        #model_obj = model(**params).fit(X_train, y_train)
        # Score do model (utilizados dados nao-balanceados) - dados de teste
        #score = recall_score(y_test, model_obj.predict(X_test))
        #scores.append(score)
    #return np.array(scores)


## Verificando melhor numero de k

### Dataset Desbalanceado

In [6]:
for k in range(2, 16):
    print('K-Fold com K = {}'.format(k))
    stratified_k_fold(df, k, random_state, shuffle=False)
    print('----'*20)

K-Fold com K = 2
k = 2, Dataset 803 positivas e 55 negativas (93.59% x 6.41%)
Fold 1: Pos: 402, Neg: 27, Total: 429, Proporção: 93.71% x 6.29%
Fold 2: Pos: 401, Neg: 28, Total: 429, Proporção: 93.47% x 6.53%
--------------------------------------------------------------------------------
K-Fold com K = 3
k = 3, Dataset 803 positivas e 55 negativas (93.59% x 6.41%)
Fold 1: Pos: 268, Neg: 18, Total: 286, Proporção: 93.71% x 6.29%
Fold 2: Pos: 268, Neg: 18, Total: 286, Proporção: 93.71% x 6.29%
Fold 3: Pos: 267, Neg: 19, Total: 286, Proporção: 93.36% x 6.64%
--------------------------------------------------------------------------------
K-Fold com K = 4
k = 4, Dataset 803 positivas e 55 negativas (93.59% x 6.41%)
Fold 1: Pos: 201, Neg: 14, Total: 215, Proporção: 93.49% x 6.51%
Fold 2: Pos: 201, Neg: 14, Total: 215, Proporção: 93.49% x 6.51%
Fold 3: Pos: 201, Neg: 13, Total: 214, Proporção: 93.93% x 6.07%
Fold 4: Pos: 200, Neg: 14, Total: 214, Proporção: 93.46% x 6.54%
-------------------

### Dataset Balanceado

In [7]:
for k in range(2, 16):
    print('K-Fold com K = {}'.format(k))
    stratified_k_fold_SMOTE(df, k, random_state, shuffle=False)
    print('----'*20)

K-Fold com K = 2
k = 2, Dataset (desbalanceado) 803 positivas e 55 negativas (93.59% x 6.41%)
	Balanceando dados de treino fold 1...
		Fold = 1, Dataset (balanceado) 397 positivas e 397 negativas (50.00% x 50.00%)
		Dados de teste (desbalanceados)
		Fold 1: Pos: 402, Neg: 27, Total: 429, Proporção: 93.71% x 6.29%
	Balanceando dados de treino fold 2...
		Fold = 2, Dataset (balanceado) 398 positivas e 398 negativas (50.00% x 50.00%)
		Dados de teste (desbalanceados)
		Fold 2: Pos: 401, Neg: 28, Total: 429, Proporção: 93.47% x 6.53%
--------------------------------------------------------------------------------
K-Fold com K = 3
k = 3, Dataset (desbalanceado) 803 positivas e 55 negativas (93.59% x 6.41%)
	Balanceando dados de treino fold 1...
		Fold = 1, Dataset (balanceado) 533 positivas e 533 negativas (50.00% x 50.00%)
		Dados de teste (desbalanceados)
		Fold 1: Pos: 268, Neg: 18, Total: 286, Proporção: 93.71% x 6.29%
	Balanceando dados de treino fold 2...
		Fold = 2, Dataset (balancea